In [11]:
!pip install -r requirements.txt

Obtaining tfx-dev from git+git://github.com/tensorflow/tfx@master#egg=tfx-dev (from -r requirements.txt (line 2))
  Cloning git://github.com/tensorflow/tfx (to revision master) to ./src/tfx-dev
  Running command git clone -q git://github.com/tensorflow/tfx /home/jupyter/ml-specialization-demo2/tfx-pipeline/src/tfx-dev
  Resolved git://github.com/tensorflow/tfx to commit 59b5e2aa573ef50b9cbee1a65792221f9f95d0d1
  Running setup.py develop for tfx-dev
    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python3.7 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/home/jupyter/ml-specialization-demo2/tfx-pipeline/src/tfx-dev/setup.py'"'"'; __file__='"'"'/home/jupyter/ml-specialization-demo2/tfx-pipeline/src/tfx-dev/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compil

In [1]:
%cd ./tfx-pipeline
GOOGLE_CLOUD_PROJECT="or2--epm-gcp-by-meetup2-t1iylu"
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
# %env GOOGLE_APPLICATION_CREDENTIALS=/Users/oleg/Projects/EPAM/creds.json
%env UNSUPPORTED_DO_NOT_PACKAGE_USER_MODULES=0


from pipeline import configs, components
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx import v1 as tfx
context = InteractiveContext(
    pipeline_root=configs.PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
              './metadata.db'
    )
)

/home/jupyter/ml-specialization-demo2/tfx-pipeline
env: GOOGLE_CLOUD_PROJECT=or2--epm-gcp-by-meetup2-t1iylu
env: UNSUPPORTED_DO_NOT_PACKAGE_USER_MODULES=0


In [2]:
from tfx.dsl.components.common.importer import Importer
examples = Importer(
    source_uri='gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/877218744686/tfx-pipeline-ml-demo2-black-friday-20210930093517/CsvExampleGen_-4992266575219261440/examples',
    artifact_type=tfx.types.standard_artifacts.Examples,
    reimport=True,
    properties={'split_names': '["train", "eval"]'}
)

context.run(examples)

examples=examples.outputs['result']
examples

Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 219
type_id: 14
uri: "gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/877218744686/tfx-pipeline-ml-demo2-black-friday-20210930093517/CsvExampleGen_-4992266575219261440/examples"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
)

In [ ]:
trainer = components.trainer_vertex(
    examples=examples,
#    schema_gen=schema_gen,
#    transform=transform
)
context.run(trainer)

In [3]:
deploy = components.pubsub_deploy()
context.run(deploy)

Publishing to PubSub topic chicago-taxi-trigger-vertex...


ExecutionResult(
    component_id: pubsub_deploy
    execution_id: 136
    outputs:
)

In [4]:
model = Importer(
    source_uri='gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/PusherVertexAI/pushed_model/124',
    artifact_type=tfx.types.standard_artifacts.Model,
    reimport=True)

context.run(model)

model=model.outputs['result']

pusher = components.pusher_vertex(
    model=model,
#    schema_gen=schema_gen,
#    transform=transform
)
context.run(pusher)

AttributeError: 'NoneType' object has no attribute 'projects'

In [5]:
!pip list | grep tfx

tfx                               1.3.0
tfx-bsl                           1.3.0
